# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import pandas as pd
import openai
import requests

openai.api_key =  "API_KEY"
openai.api_base = "https://openai.vocareum.com/v1" # Remove this if using personal key


params = {
    "action": "query", 
    "prop": "extracts",
    "exlimit": 1,
    "titles": "Synthesizer",
    "explaintext": 1,
    "formatversion": 2,
    "format": "json"
}
resp = requests.get("https://en.wikipedia.org/w/api.php", params=params)
response_dict = resp.json()

In [2]:
#response_dict 
text_data = response_dict["query"]["pages"][0]["extract"].split("\n")
#leaving older code that was used for clean up above for learnings.
#response = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&rvprop=content&titles=Synthesizer&rvslots=*")
#response.json()["query"]["pages"]["10791746"]["revisions"][0]["slots"]["main"]["*"].split("\n")

In [3]:
# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = text_data
df

,text
0,A synthesizer (also synthesiser or synth) is a...
1,
2,Synthesizer-like instruments emerged in the Un...
3,"In 1978, Sequential Circuits released the Prop..."
4,Synthesizers were initially viewed as avant-ga...
...,...
157,== External links ==
158,
159,Sound Synthesis Theory wikibook
160,Principles of Sound Synthesis Archived 20 Janu...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [4]:
# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

In [5]:
df
# Save to CSV
df.to_csv('text.csv', index=False)

In [6]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
response = openai.Embedding.create(
    input=df["text"].tolist(),
    model=EMBEDDING_MODEL_NAME
)

# Extract and print the first 20 numbers in the embedding
response_list = response["data"]
first_item = response_list[0]
first_item_embedding = first_item["embedding"]
print(first_item_embedding[:20])
len(first_item_embedding)

[-0.02433592826128006, -0.014530242420732975, -0.019420349970459938, -0.007583487778902054, -0.01973871700465679, 0.023228012025356293, -0.028984077274799347, 0.0027745631523430347, -0.02641167677938938, -0.004559007007628679, 0.028067180886864662, 0.022820502519607544, -0.026946531608700752, 0.0012432207586243749, 0.014479303732514381, -0.0007987017161212862, 0.0294679943472147, -0.009792950004339218, 0.0071632438339293, -0.03125084564089775]


1536

In [7]:
embeddings = [data["embedding"] for data in response["data"]]
embeddings

[[-0.02433592826128006,
  -0.014530242420732975,
  -0.019420349970459938,
  -0.007583487778902054,
  -0.01973871700465679,
  0.023228012025356293,
  -0.028984077274799347,
  0.0027745631523430347,
  -0.02641167677938938,
  -0.004559007007628679,
  0.028067180886864662,
  0.022820502519607544,
  -0.026946531608700752,
  0.0012432207586243749,
  0.014479303732514381,
  -0.0007987017161212862,
  0.0294679943472147,
  -0.009792950004339218,
  0.0071632438339293,
  -0.03125084564089775,
  -0.021547038108110428,
  0.006080798339098692,
  -0.00782544631510973,
  -0.006354593206197023,
  -0.02529102750122547,
  0.018618067726492882,
  0.0014024039264768362,
  -0.02155977301299572,
  -0.015421668067574501,
  0.01959863491356373,
  0.02026083692908287,
  0.007004060782492161,
  -0.027455918490886688,
  -0.0088887894526124,
  -0.018516190350055695,
  0.0004496925394050777,
  -0.0005539575358852744,
  -0.023075196892023087,
  -0.005358106456696987,
  -0.008328464813530445,
  0.025252822786569595,


In [8]:
# Add embeddings list to dataframe
df["embeddings"] = embeddings

/tmp/ipykernel_62/3663513068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["embeddings"] = embeddings


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2